# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`
mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'fruits_db', 'local', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [26]:
# review a document in the customer_list collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658237798112a47b0e2cbc6e'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [25]:
# assign the collection to a variable
establishments = db['establishments']

In [28]:
print("Number of documents in result:", establishments.count_documents({}))


Number of documents in result: 39779


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [29]:
# Create a dictionary for the new restaurant data
new_establishment = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [30]:
# Insert the new restaurant into the collection
establishments.insert_one(new_establishment)

In [31]:
print("Number of documents in result:", establishments.count_documents({}))

Number of documents in result: 39780


In [33]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishments.find(query)

pprint(results[0])

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65823ccf957d7e61fa4adc88'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [35]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType" : "Restaurant/Cafe/Canteen", "BusinessTypeID" : {"$ne": ''}}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}

results = establishments.find(query, fields)
pprint(results[0])


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('658237798112a47b0e2cbc6f')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [37]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {'$set':
        {'BusinessTypeID': 1}
    }
)

In [39]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {'BusinessName': 1, 'BusinessTypeID': 1, 'BusinessType': 1}
results = establishments.find(query, fields)

pprint(results[0])

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65823ccf957d7e61fa4adc88')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [41]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName' : 'Dover'}
print("Number of documents in result:", establishments.count_documents(query))

Number of documents in result: 994


In [43]:
# Delete all documents where LocalAuthorityName is "Dover"
results = establishments.delete_many(query)

print(f"Deleted {results.deleted_count} documents")

Deleted 994 documents


In [44]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName' : 'Dover'}
print("Number of documents in result:", establishments.count_documents(query))

Number of documents in result: 0


In [48]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Folkestone Harbour',
 'AddressLine2': 'Harbour Approach Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'Dr Legumes - Harbour Arm',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.914705329384,
 'FHRSID': 805702,
 'LocalAuthorityBusinessID': 'PI/000073219',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': '',
 'RatingDate': '2018-06-08T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658237798112a47b0e2cbf54'),
 'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/805702',
            'rel': 'self'}],
 'me

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [47]:
# Change the data type from String to Decimal for longitude and latitude
results = establishments.update_many({}, [ {'$set': {'geocode.latitude' : {'$toDouble' : '$geocode.latitude'}, 
                                           'geocode.longitude' : {'$toDouble' : '$geocode.longitude'} }
                                  } ]
                            )

print(f"Updated {results.modified_count} documents")


Updated 38786 documents


Use `update_many` to convert `RatingValue` to integer numbers.

In [49]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
results = establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

print(f"Updated {results.modified_count} documents")


Updated 0 documents


In [50]:
# Change the data type from String to Integer for RatingValue
results = establishments.update_many({}, [ {'$set' : {'RatingValue' : {'$toInt' : '$RatingValue'}}}])

print(f"Updated {results.modified_count} documents")

Updated 34695 documents


In [51]:
# Check that the coordinates and rating value are now numbers
pprint(db.establishments.find_one())

{'AddressLine1': 'Folkestone Harbour',
 'AddressLine2': 'Harbour Approach Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'Dr Legumes - Harbour Arm',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.914705329384,
 'FHRSID': 805702,
 'LocalAuthorityBusinessID': 'PI/000073219',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': '',
 'RatingDate': '2018-06-08T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658237798112a47b0e2cbf54'),
 'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/805702',
            'rel': 'self'}],
 'meta